<a href="https://colab.research.google.com/github/mrlujanero/03-MAIR--Algoritmos-De-Optimizacion--2019/blob/master/SEMINARIOS/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Ramón Luján Crespo:   <br>
Url: https://github.com/mrlujanero/03-MAIR--Algoritmos-De-Optimizacion--2019/tree/master/SEMINARIOS<br>
Problema:
> 1. Elección de grupos de población homogéneos <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema: Organizar los horarios de partidos de La Liga

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>





Respuesta: Para este problema la restricción es que se repitan o no los horarios de los partidos. Si se tiene en cuenta que se puedan repetir horarios de partidos, hay un total de:
solucionesPosibles = horariosPosibles ^ numeroPartidos = 10^10 = 10000000000
En caso de que no se pudieran repetir horarios, entonces sería un problema de permutaciones, por lo que habría un total de numeroPartidos! = 10! = 3628800

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta: El espacio de soluciones que se ha elegido consiste en un array donde cada posición del mismo representa un partido, y cada elemento en la posición i contiene el horario en el que se va a jugar dicho partido. Se ha considerado que esta es la mejor manera de representar la solución, ya que de este modo se tiene un único array de una sola dimensión. Otra opción sería una estructura donde cada posición representase un horario, y cada elemento fuera una lista de los partidos que se celebran  ese día, pero se ha considerado que era más complejo trabajar con ella.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta: 
La función objetivo consiste en el cálculo de la audiencia según la tabla de estadísticas de audiencia que se genera en cada partido, junto con el día que se celebra el partido. Además, se debe tener en cuenta si se celebran varios partidos en el mismo momento. 

Se trata de un problema de maximización, ya que lo que se persigue es conseguir la aquella distribución de partidos que mejor audiencia tenga.

In [0]:
def calcularAudienciaSiEsSolucion(audiencia, horarios):
    for dia in horarios.keys():
        partidosPorHorario = horarios[dia]["partidosPorHorario"]
        if len(partidosPorHorario) > 1:
            maxPuntos = horarios[dia]["puntosPorHorario"]
            maxAudiencia = horarios[dia]["maxAudienciaHorario"]
            for (equipos, puntos) in partidosPorHorario:
                audiencia += (puntos / maxPuntos) * maxAudiencia
            audiencia = audiencia * estadisticasHorarios[int(dia)]
        elif len(partidosPorHorario) == 1:
            equipoA = partidosPorHorario[0][0][0]
            equipoB = partidosPorHorario[0][0][1]
            audiencia += tablaAudiencias[equipoA][equipoB] * estadisticasHorarios[int(dia)]
    return audiencia

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [0]:
#descargar pdf nuevo
import random

equipoCategoria = ["A","A","A","B","B","B","B","C","C","C","A","A","A","B","B","B","B","C","C","C"]


puntosCategoria = {
    "A": 4,
    "B": 2,
    "C": 1
}

tablaAudiencias = {
    "A": {"A": 2, "B": 1.3, "C": 1},
    "B": {"A": 1.3, "B": 0.9, "C": 0.75},
    "C": {"A": 1, "B": 0.75, "C": 0.47}
}

estadisticasHorarios = [0.4,0.55,0.7,0.8,1,0.45,0.75,0.85,1,0.4]

def generarPartidosAleatorios():
    partidos = []
    equipos = list(equipoCategoria)
    random.shuffle(equipos)
    numPartidos = int(len(equipos)/2)
    for i in range(numPartidos):
        equipoA = equipos[i]
        equipoB = equipos[i+numPartidos]
        puntos = puntosCategoria[equipoA] + puntosCategoria[equipoB]
        partidos.append(((equipos[i],equipos[i+numPartidos]),puntos))
    print(partidos)
    return partidos


def organizarPartidosFuerzaBruta():
    partidos = generarPartidosAleatorios()
    mejorSolucion = []
    mejorAudiencia = 0
    for combinacion in range(0, len(estadisticasHorarios)**len(equipoCategoria)):
        horario = list('{0:010}'.format(combinacion))
        audiencia = calcularAudiencias(partidos, horario)
        if audiencia > mejorAudiencia:
            mejorAudiencia = audiencia
            mejorSolucion = horario
    return mejorSolucion, mejorAudiencia

def esSolucion(horario):
    hayEnViernes = False
    hayEnLunes = False
    for i in range(len(horario)):
        if horario[i] == '0':
            hayEnViernes = True
        elif horario[i] == '9':
            hayEnLunes = True
    return hayEnViernes and hayEnLunes

def calcularAudiencias(partidos, horario):
    audiencia = 0
    horarios = {}
    #si hay al menos un partido en viernes y al menos uno en lunes
    if esSolucion(horario):
        organizarPartidosPorDiccionarioHorario(horario, horarios, partidos)
        audiencia = calcularAudienciaSiEsSolucion(audiencia, horarios)
    return audiencia

#calcula las audiencias para la combinacion candidata
def calcularAudienciaSiEsSolucion(audiencia, horarios):
    for dia in horarios.keys():
        partidosPorHorario = horarios[dia]["partidosPorHorario"]
        #si mas de un partido cae en el mismo horario
        if len(partidosPorHorario) > 1:
            audiencia = calcularAudienciaPartidosCoincidentes(audiencia, dia, horarios, partidosPorHorario)
        #si solo hay un partido en este horario
        elif len(partidosPorHorario) == 1:
            audiencia = calcularAudienciaPartidoUnico(audiencia, dia, partidosPorHorario)
    return audiencia


def calcularAudienciaPartidoUnico(audiencia, dia, partidosPorHorario):
    equipoA = partidosPorHorario[0][0][0]
    equipoB = partidosPorHorario[0][0][1]
    audiencia += tablaAudiencias[equipoA][equipoB] * estadisticasHorarios[int(dia)]
    return audiencia


def calcularAudienciaPartidosCoincidentes(audiencia, dia, horarios, partidosPorHorario):
    maxPuntos = horarios[dia]["puntosPorHorario"]
    maxAudiencia = horarios[dia]["maxAudienciaHorario"]
    for (equipos, puntos) in partidosPorHorario:
        audiencia += (puntos / maxPuntos) * maxAudiencia
    audiencia = audiencia * estadisticasHorarios[int(dia)]
    return audiencia


#crea un diccionario donde cada clave es un horario, la cual contiene una lista de partidos, la audiencia maxima para cada hora y los puntos de cada partid
#para ser usados en caso de que coincidan varios partidos en el mismo dia
def organizarPartidosPorDiccionarioHorario(horario, horarios, partidos):
    for i in range(len(partidos)):
        dia = horario[i]
        if dia not in horarios:
            anadirDiaADiccionarioHorario(dia, horarios)
        anadirPartidoADia(dia, horarios, i, partidos)


def anadirPartidoADia(dia, horarios, i, partidos):
    horarios[dia]["puntosPorHorario"] += partidos[i][1]
    horarios[dia]["partidosPorHorario"].append(partidos[i])
    if horarios[dia]["maxAudienciaHorario"] < tablaAudiencias[partidos[i][0][0]][partidos[i][0][1]]:
        horarios[dia]["maxAudienciaHorario"] = tablaAudiencias[partidos[i][0][0]][partidos[i][0][1]]


def anadirDiaADiccionarioHorario(dia, horarios):
    horarios[dia] = {}
    horarios[dia]["puntosPorHorario"] = 0
    horarios[dia]["partidosPorHorario"] = []
    horarios[dia]["maxAudienciaHorario"] = 0


#organizarPartidosFuerzaBruta()

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta
(horariosPosibles ^ numPartidos) * horariosPosibles

(horariosPosibles ^ numPartidos) -> puede verse en el bucle principal que controla el algoritmo: for combinacion in range(0, len(estadisticasHorarios)**len(equipoCategoria)). Este bucle genera todas las soluciones posibles desde [0,0,...,0] hasta [9,9,...,9].

El siguiente bucle del que se extrae horarios se puede encontrar al calcular las audiencias. Se recorren los dias posibles, y dentro de cada dia sus partidos. En total se sabe que existen tantos partidos como horarios disponibles, por lo que ya sea el caso de que coincidan todos los partidos en el mismo dia, o todos los partidos repartidos, al final el número de vueltas será el mismo, el número de horarios posible o el número de partidos posibles.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta: Para este caso se plantea una aproximación voraz. Para ello, se ordenan los partidos según su audiencia, y se ordenan sus horarios según su multiplicador de la audiencia, para seguir la idea de que el partido que mejor audiencia tenga, debe tener el mejor multiplicador.
Mejora el algoritmo de fuerza bruta, ya que en vez de probar todas las soluciones para obtener sus audiencias, desde el principio se ha organizado una solución óptima.

Tan sólo ha habido que modificar la función para calcular las audiencias, ya que no es necesario comprobar si hay un partido en lunes y otro en viernes. Además, en este caso ya no ocurre que  en un mismo horario ocurran diferentes partidos, lo cual además afectaba negativamente a la audiencia.

In [5]:
def organizarPartidosVoraz():
    partidos = generarPartidosAleatorios()
    partidosOrganizadosPorPuntos = sorted(partidos, key = lambda partido: partido[1], reverse=True)
    estadisticasOrdenadas = sorted(range(len(estadisticasHorarios)), key=lambda k: estadisticasHorarios[k], reverse = True)
    audiencia = calcularAudiencias(partidosOrganizadosPorPuntos, estadisticasOrdenadas)
    return partidos, estadisticasOrdenadas, audiencia
  
def calcularAudiencias(partidos, horario):
    audiencia = 0
    for i in range(len(partidos)):
      equipoA = partidos[i][0][0]
      equipoB = partidos[i][0][1]
      audiencia += tablaAudiencias[equipoA][equipoB] * estadisticasHorarios[horario[i]]
    return audiencia
  
def calcularAudienciaSiEsSolucion(audiencia, horarios):
    for dia in horarios.keys():
        partidosPorHorario = horarios[dia]["partidosPorHorario"]
        audiencia = calcularAudienciaPartidoUnico(audiencia, dia, partidosPorHorario)
    return audiencia
  
organizarPartidosVoraz()

[(('C', 'B'), 3), (('B', 'B'), 4), (('A', 'C'), 5), (('A', 'A'), 8), (('A', 'A'), 8), (('C', 'B'), 3), (('B', 'B'), 4), (('C', 'C'), 2), (('B', 'C'), 3), (('B', 'A'), 6)]


([(('C', 'B'), 3),
  (('B', 'B'), 4),
  (('A', 'C'), 5),
  (('A', 'A'), 8),
  (('A', 'A'), 8),
  (('C', 'B'), 3),
  (('B', 'B'), 4),
  (('C', 'C'), 2),
  (('B', 'C'), 3),
  (('B', 'A'), 6)],
 [4, 8, 7, 3, 6, 2, 1, 5, 0, 9],
 8.448)

(*)Calcula la complejidad del algoritmo 

Respuesta: Tal y como he encontrado en este enlace https://stackoverflow.com/questions/14434490/what-is-the-complexity-of-this-python-sort-method, se dice que los métodos de ordenación propios de python tienen una complejidad de n*log(n).

De este modo, la ordenación de ambas listas tiene una complejidad de 2*n*log(n), siendo n el número de partidos y el número de horarios, que en este problema coinciden.
Por otro lado, para calcular las audiencias se puede ver un bucle con complejidad 3n.

De este modo, la complejidad de este algoritmo es 2n*log(n)+3n

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta: Para este problema se están generando aleatoriamente los partidos.

In [0]:
def generarPartidosAleatorios():
    partidos = []
    equipos = list(equipoCategoria)
    random.shuffle(equipos)
    numPartidos = int(len(equipos)/2)
    for i in range(numPartidos):
        equipoA = equipos[i]
        equipoB = equipos[i+numPartidos]
        puntos = puntosCategoria[equipoA] + puntosCategoria[equipoB]
        partidos.append(((equipos[i],equipos[i+numPartidos]),puntos))
    print(partidos)
    return partidos

Aplica el algoritmo al juego de datos generado: Ya se está aplicando.

Respuesta

Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta